In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import pandas as pd

# Weather API Call

In [21]:

airport_geo_df = pd.read_csv('/Users/conorjohnston/code/diego-mogollon/lateguru/lateguru/raw_data/Dataset_A/Dataset_A_US2023_Kaggle_airports_geolocation.csv')

airport_geo_df.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [22]:
cleaned_airport_geos_df = airport_geo_df[['IATA_CODE', 'LATITUDE', 'LONGITUDE']]\
    .rename(columns={'IATA_CODE': 'airport_acronym',
                     'LATITUDE': 'lat',
                     'LONGITUDE':'lon'})

In [23]:
cleaned_airport_geos_df.head()

,airport_acronym,lat,lon
0,ABE,40.65236,-75.44040
1,ABI,32.41132,-99.68190
2,ABQ,35.04022,-106.60919
3,ABR,45.44906,-98.42183
4,ABY,31.53552,-84.19447


In [24]:
airport='LAX'

In [85]:
def get_weather_data(lat, lon):
    #lat= -37.663712
    #lon= 144.844788


    api_key='9efa0a3dd9883bae2b9f3bda6eff2e36'

    url = "https://api.openweathermap.org/data/2.5/weather?"
    params = {'lat': str(lat), 'lon':str(lon), "appid":api_key}
    response = requests.get(url, params=params)
    json_response = response.json()

    print(json_response)

    df = pd.DataFrame({
    "temp": json_response['main']['temp'],
    "feels_like_temp": json_response['main']['feels_like'],
    "alt_pressure": json_response['main']['pressure'],
    "sl_pressure": json_response['main']['sea_level'],
    "visibility": json_response['visibility'],
    "wind_speed": json_response['wind']['speed'],
    "wind_gust": json_response['wind']['deg']
    #precipitation and ice accretion to be added
    }, index = [0])

    #clean up json response object before outputting data
    return df

In [84]:
api_key='9efa0a3dd9883bae2b9f3bda6eff2e36'

url = "https://api.openweathermap.org/data/2.5/weather?"
params = {'lat': str(lat), 'lon':str(lon), "appid":api_key}
response = requests.get(url, params=params)
json_response = response.json()
json_response['main']

{'temp': 290.47,
 'feels_like': 290.58,
 'temp_min': 289.19,
 'temp_max': 291.99,
 'pressure': 1016,
 'humidity': 89,
 'sea_level': 1016,
 'grnd_level': 1011}

In [26]:
def get_lat_lon_cordinates(airport):
    '''Function that takes in an airport variable that is a string of the airports IATA code acronym and outputs
    the geo data on this airport as a dictionary'''

    selected_airport = cleaned_airport_geos_df[cleaned_airport_geos_df['airport_acronym']==airport]

    airport_geo_codes = {
        #'name': selected_airport['airport_acronym'],
        'lat': selected_airport['lat'],
        'lon': selected_airport['lon']
    }


    return selected_airport['lat'].values[0], selected_airport['lon'].values[0]

In [27]:
selected_airport = cleaned_airport_geos_df[cleaned_airport_geos_df['airport_acronym']==airport]

selected_airport['lon'].values[0]

-118.40807

In [31]:
cleaned_airport_geos_df = airport_geo_df[['IATA_CODE', 'LATITUDE', 'LONGITUDE']]\
    .rename(columns={'IATA_CODE': 'airport_acronym',
                     'LATITUDE': 'lat',
                     'LONGITUDE':'lon'})

In [32]:
lat, lon = get_lat_lon_cordinates(airport)


In [86]:
X_weather = get_weather_data(lat, lon)

{'coord': {'lon': -118.4081, 'lat': 33.9425}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 290.5, 'feels_like': 290.64, 'temp_min': 289.19, 'temp_max': 291.85, 'pressure': 1016, 'humidity': 90, 'sea_level': 1016, 'grnd_level': 1011}, 'visibility': 10000, 'wind': {'speed': 2.06, 'deg': 60}, 'clouds': {'all': 0}, 'dt': 1724585802, 'sys': {'type': 1, 'id': 4667, 'country': 'US', 'sunrise': 1724592180, 'sunset': 1724639311}, 'timezone': -25200, 'id': 5345860, 'name': 'El Segundo', 'cod': 200}


In [82]:
X_weather['temp'].values[0]

291.04

NameError: name 'json_response' is not defined

# Building the FAST API

In [51]:
import pandas as pd
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import os
import joblib

import numpy as np
from sklearn.decomposition import PCA

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer


In [74]:
model_directory = '/Users/conorjohnston/code/diego-mogollon/lateguru/lateguru/model'

def load_model():
    filepath = '/Users/conorjohnston/code/diego-mogollon/lateguru/lateguru/model/_model.pkl'
    model = joblib.load(filepath)
    print(f"Model loaded from {filepath}")
    return model

def create_sklearn_preprocessor():

    categorical_features = ['Origin', 'Dest', 'Route', 'Carrier']

    numerical_features = [
        'DayOfWeek', 'HourOfDay', 'Temperature', 'Feels_Like_Temperature',
        'Altimeter_Pressure', 'Sea_Level_Pressure', 'Visibility',
        'Wind_Speed', 'Wind_Gust', 'Precipitation', 'Ice_Accretion_3hr',
        'CarrierAvgDelay', 'Month'
    ]

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])


    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])


    final_preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        n_jobs=-1
    )

    return final_preprocessor

def preprocess_features_v2(X) :

    preprocessor = create_sklearn_preprocessor()

    X_processed = preprocessor.fit_transform(X)

    X_processed = pd.DataFrame(X_processed, columns=X.columns)

    # print(f"✅ X_processed, with shape {X_processed.shape}")

    return X_processed

def model_predict(model, X_pred):
    y_pred = model.predict(X_pred)
    return y_pred

In [75]:
preprocessor = create_sklearn_preprocessor()

X_processed = preprocessor.fit_transform(X_pred)

X_processed = pd.DataFrame(X_processed, columns=X_pred.columns)

# print(f"✅ X_processed, with shape {X_processed.shape}")
X_processed

,Origin,Dest,Route,Carrier,DayOfWeek,HourOfDay,Temperature,Feels_Like_Temperature,Altimeter_Pressure,Sea_Level_Pressure,Visibility,Wind_Speed,Wind_Gust,Precipitation,Ice_Accretion_3hr,CarrierAvgDelay,Month
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0


In [40]:
app = FastAPI()

In [41]:
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allows all origins
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods
    allow_headers=["*"],  # Allows all headers
)

In [77]:
@app.get("/predict")
def predict(
    origin: str, #Origin airport code e.g JFK
    destination: str, #Destination airport code e.g SFO
    carrier: str, #name of airline carrier
    hour: float, #hour at time of flight departure
    day_of_week: float, #number corresponding to day of week
    month: float, #month of year at time of departure

    #delayed: bool, #True if flight delayed, else False (default)
    #delay_len: float, #amount of mins flight is delayed
    #carrier_avg_delay: float, #amount of mins on average that each carrier is delayed when delayed
    #temperature: float, #temperature in origin at time of prediction
    #feels_like_temp: float, #feels like temperature in origin at time of prediction
    #alt_pressure: float, #air pressure at origin
    #sl_pressure: float, #sea level pressure at origin
    #visibility: float, #measure of how clear visbility at origin is
    #wind_speed: float, #measure of wind speed in km/h
    #wind_gust: float, #measure of wind gust strength at origin
    #precipitation: float, #measure of rainfall at origin
    #ice_accretion_3hr: float, #measure of ice at origin
    #route: str, #concatenation of origin and destination airport string
):
    """Make a prediction based on inputs as to whether the flight is likely
    to be delayed by weather
    Assumes flight is taking place in the USA"""



    X_pred = pd.DataFrame({
        "Origin": [str(origin)],
        "Dest": [str(destination)],
        "Route": [str('LAX_JFK')],
        "Carrier": [str(carrier)],

        #"delay_len": [float(delay_len)],
        #'''"temperature": [float(X_weather['main']['temp'])],
        #"feels_like_temp": [float(X_weather['main']['feels_like'])],
        #"alt_pressure": [float(X_weather['main']['pressure'])],
        #"sl_pressure": [float(X_weather['main']['sea_level'])],
        #"visibility": [float(X_weather['visibility'])],
        #"wind_speed": [float(X_weather['wind']['speed'])],
        #"wind_gust": [float(X_weather['wind']['gust'])],'''

        "DayOfWeek": [float(day_of_week)],
        "HourOfDay": [float(hour)],
        "Temperature": [float(32)],
        "Feels_Like_Temperature": [float(14)],
        "Altimeter_Pressure": [float(3478)],
        "Sea_Level_Pressure": [float(1293)],
        "Visibility": [float(1067)],
        "Wind_Speed": [float(15)],
        "Wind_Gust": [float(5)],
        #must still input precipitation
        "Precipitation": [float(0.0)],
        #must still input ice accretion
        "Ice_Accretion_3hr": [float(0.0)],
        "CarrierAvgDelay": [float(20)],
        "Month": [float(month)]
        })

    model = load_model()

    X_processed = preprocess_features_v2(X_pred)


    y_pred = model_predict(model, X_processed)

    return dict(likely_to_be_delayed=bool(y_pred))

In [70]:
X_pred = pd.DataFrame({
        "Origin": [str('LAX')],
        "Dest": [str('JFK')],
        "Route": [str('LAX_JFK')],
        "Carrier": [str('American Airlines Inc.')],

        #"delay_len": [float(delay_len)],
        #'''"temperature": [float(X_weather['main']['temp'])],
        #"feels_like_temp": [float(X_weather['main']['feels_like'])],
        #"alt_pressure": [float(X_weather['main']['pressure'])],
        #"sl_pressure": [float(X_weather['main']['sea_level'])],
        #"visibility": [float(X_weather['visibility'])],
        #"wind_speed": [float(X_weather['wind']['speed'])],
        #"wind_gust": [float(X_weather['wind']['gust'])],'''

        "DayOfWeek": [float(4)],
        "HourOfDay": [float(20)],
        "Temperature": [float(32)],
        "Feels_Like_Temperature": [float(14)],
        "Altimeter_Pressure": [float(3478)],
        "Sea_Level_Pressure": [float(1293)],
        "Visibility": [float(1067)],
        "Wind_Speed": [float(15)],
        "Wind_Gust": [float(5)],
        #must still input precipitation
        "Precipitation": [float(0.0)],
        #must still input ice accretion
        "Ice_Accretion_3hr": [float(0.0)],
        "CarrierAvgDelay": [float(20)],
        "Month": [float(7)]
        })

type(X_pred)

pandas.core.frame.DataFrame

In [79]:
X_pred

,Origin,Dest,Route,Carrier,DayOfWeek,HourOfDay,Temperature,Feels_Like_Temperature,Altimeter_Pressure,Sea_Level_Pressure,Visibility,Wind_Speed,Wind_Gust,Precipitation,Ice_Accretion_3hr,CarrierAvgDelay,Month
0,LAX,JFK,LAX_JFK,American Airlines Inc.,4.0,20.0,32.0,14.0,3478.0,1293.0,1067.0,15.0,5.0,0.0,0.0,20.0,7.0


In [78]:
predict('LAX', 'JFK', 'American Airlines Inc.', '10', '4', '4')

Model loaded from /Users/conorjohnston/code/diego-mogollon/lateguru/lateguru/model/_model.pkl


{'likely_to_be_delayed': True}